In [2]:
import json
from tqdm import tqdm_notebook
import operator
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
from rdkit.Chem import rdFMCS

LEVEL1 = set(["C", "H", "O", "N", "c", "h", "o", "n"])
LEVEL2 = set(["Br", "S", "Cl", "B", "F", "br", "s", "cl", "b", "f"])
LEVEL3 = set(["Si", "I", "P", "si", "i", "p"])
LEVEL4 = set(["Sn", "Se", "Mg", "sn", "se", "mg"])

In [2]:
reactions = []
with open('reactions_02_10_2019.json') as fstream:
    for line in fstream:
        reactions.append(json.loads(line))

In [7]:
def is_one_to_one(reaction: dict) -> bool:
    return len(reaction['reactants']) == 1 and len(reaction['products']) == 1


def is_two_to_one(reaction: dict) -> bool:
    return len(reaction['reactants']) == 2 and len(reaction['products']) == 1


def filter_reactions(reactions: list) -> list:
    """
    Отбирает реакции 1 реагент -> 1 продукт; 2 реагента -> 2 продукта
    Из каждой группы дубликатов берет только одну реакцию, первую встреченную
    
    reactions - список реакций, где каждый элемент - словарик реакции
    """
    ids = set()
    filtered = []
    for reaction in reactions:
        if reaction['reaxys_id'] not in ids:
            if is_one_to_one(reaction) or is_two_to_one(reaction):
                filtered.append(reaction)
                ids.add(reaction['reaxys_id'])
    return filtered


def find_duplicated(reactions: list) -> dict:
    """
    Находит группы дубликатов, возвращает словарь 
    """
    from collections import defaultdict
    r_groups = defaultdict(list)
    for reaction in reactions:
        r_groups[reaction['reaxys_id']].append(reaction)
    return {k: v for k, v in r_groups.items() if len(v) > 1}


def to_one_json(reactions: list, p_key: str='reaxys_id') -> dict:
    """
    Преобразует список реакций в один словарь, который можно сохранить
    в json. Используются ключ 'reaxys_id'
    """
    result = {}
    for reaction in reactions:
        curr_key = reaction[p_key]
        del reaction[p_key]
        result[p_key] = reaction
    return result

def draw_mol(mol):
    """
    Рисует молекулу
    """
    AllChem.Compute2DCoords(mol)
    return Draw.MolToImage(mol)


def get_atomic_vocab(reactions: list) -> (dict, list):
    from collections import defaultdict
    cant_parse = []
    res = defaultdict(int)
    for reaction in tqdm_notebook(reactions):
        lhs, rhs = reaction['reaction_core'].split('>>')
        lhs = Chem.MolFromSmarts(lhs)
        rhs = Chem.MolFromSmarts(rhs)
        if lhs is None or rhs is None:
            cant_parse.apend(reaction)
        for atom in lhs.GetAtoms():
            res[atom.GetSymbol()] += 1
        for atom in rhs.GetAtoms():
            res[atom.GetSymbol()] += 1
    return res, cant_parse


def get_reaction_atomic_vocab(reactions: list) -> (dict, list):
    from collections import defaultdict
    cant_parse = []
    res = defaultdict(int)
    for reaction in tqdm_notebook(reactions):
        for reactant in (reaction['reactants'] + reaction['products']):
            reactant = Chem.MolFromSmarts(reactant)
            if reactant is None:
                cant_parse.append(reaction)
                break
            for atom in reactant.GetAtoms():
                res[atom.GetSymbol()] += 1
    return res, cant_parse


def are_equal(mol1, mol2):
    """
    Равенство для двух молекул
    """
    mols = [mol1, mol2]
    n_atoms1, n_atoms2 = map(lambda x: x.GetNumAtoms(), mols)
    n_bonds1, n_bonds2 = map(lambda x: x.GetNumBonds(), mols)
    if n_atoms1 != n_atoms2 or n_bonds1 != n_bonds2:
        return False
    res = rdFMCS.FindMCS(mols)
    if n_atoms1 == res.numAtoms and n_bonds1 == res.numBonds:
        return True
    return False

In [4]:
filtered = filter_reactions(reactions)

In [7]:
vocab, cantparse = get_atomic_vocab(filtered)

In [8]:
print(sorted(vocab.items(), key=operator.itemgetter(1), reverse=True))

[('C', 5867840), ('H', 4023211), ('O', 1268220), ('N', 768823), ('Br', 97035), ('S', 89774), ('Cl', 66620), ('B', 66356), ('F', 56382), ('Si', 22699), ('I', 13310), ('P', 11363), ('Sn', 1943), ('Se', 1390), ('Mg', 914), ('As', 307), ('Sb', 226), ('Li', 196), ('Te', 194), ('Ge', 156), ('Zn', 155), ('Au', 145), ('Cu', 115), ('Cr', 106), ('Ni', 79), ('Hg', 71), ('W', 71), ('Ti', 57), ('Co', 55), ('Al', 49), ('Pb', 40), ('Pd', 35), ('Ir', 35), ('Na', 29), ('Bi', 27), ('Zr', 26), ('Mn', 22), ('Rh', 22), ('Cd', 19), ('Tl', 14), ('Mo', 13), ('*', 12), ('Ag', 11), ('K', 10), ('Ce', 10), ('Ga', 9), ('Ru', 8), ('Re', 8), ('Fe', 7), ('In', 7), ('V', 7), ('Hf', 4), ('U', 4), ('Os', 3), ('Tc', 2), ('Pr', 2), ('Ta', 2), ('Nd', 1)]


In [9]:
len(cantparse)

0

In [110]:
# Средняя длина
np.mean([len(reaction['reaction_core']) for reaction in filtered])

113.84371441836097

In [8]:
rvocab, cantparse_r = get_reaction_atomic_vocab(filtered)

In [9]:
len(cantparse_r)

0

In [10]:
print(sorted(rvocab.items(), key=operator.itemgetter(1), reverse=True))

[('H', 19278461), ('C', 16681109), ('O', 2849947), ('N', 1718850), ('F', 275624), ('S', 188127), ('Cl', 166943), ('Br', 136031), ('B', 63795), ('Si', 46388), ('I', 18103), ('P', 17814), ('Se', 2367), ('Sn', 2102), ('Mg', 786), ('As', 341), ('Sb', 218), ('Te', 214), ('Zn', 185), ('Li', 168), ('Ge', 163), ('Au', 135), ('Cu', 114), ('Cr', 106), ('Ni', 93), ('Ir', 76), ('W', 75), ('Hg', 67), ('Co', 60), ('Ti', 57), ('Al', 44), ('Pb', 38), ('Pd', 35), ('*', 34), ('Na', 28), ('Zr', 26), ('Bi', 25), ('Mn', 22), ('Rh', 22), ('Ce', 18), ('Cd', 18), ('Mo', 13), ('Tl', 12), ('K', 10), ('Ga', 9), ('Ru', 8), ('Re', 8), ('In', 7), ('Ag', 7), ('V', 7), ('Fe', 6), ('Hf', 4), ('U', 4), ('Os', 3), ('Tc', 2), ('Nb', 2), ('Pr', 2), ('Ta', 2), ('Nd', 1)]


In [79]:
def smi_tokenizer(smi):
    """
    Tokenize a SMILES molecule or reaction
    
    Что я изменил:
    1. добавил поиск H: "...|H|..."
    2. "...|S|..." -> "...|Si?|..."
    3. "...|>|..." -> "...|>>|...", символ реакции - один токен
    
    * У них есть такой паттерн \[[^\]]+], то есть все, что обернуто в квадратные скобки - один токен,
    видел такое и в других статьях. Не очень понимаю только, почему паттерн не \[[^\]]+\]. Разве не нужно
    экранировать вторую закрывающую скобку тоже? На всякий случай, я экранировал вторую скобку тоже.
    """
    import re
    pattern =  "(\[[^\]]+]|Br?|Cl?|H|N|O|Si?|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\\\|\/|:|~|@|\?|>>|\*|\$|\%[0-9]{2}|[0-9])"
    regex = re.compile(pattern)
    tokens = [token for token in regex.findall(smi)]
    #assert smi == ''.join(tokens)
    return ' '.join(tokens)

In [12]:
def get_atom_set(mol) -> set:
    res = set()
    for atom in mol.GetAtoms():
        res.add(atom.GetSymbol())
    return res


def filter_by_atoms(reactions: list, atom_set: set) -> list:
    filtered = []
    for reaction in tqdm_notebook(reactions):
        lhs, rhs = reaction['reaction_core'].split('>>')
        lhs = Chem.MolFromSmarts(lhs)
        rhs = Chem.MolFromSmarts(rhs)
        lhs_set = get_atom_set(lhs)
        rhs_set = get_atom_set(rhs)
        if (lhs_set | rhs_set) <= atom_set:
            filtered.append(reaction)
    return filtered


def filter_reactions_by_atoms(reactions: list, atom_set: set) -> list:
    filtered = []
    for reaction in tqdm_notebook(reactions):
        curr_set = set()
        for reagent in (reaction['reactants'] + reaction['products']):
            tmp = get_atom_set(Chem.MolFromSmarts(reagent))
            curr_set = curr_set | tmp
        if curr_set <= atom_set:
            filtered.append(reaction)
    return filtered

In [211]:
chno = filter_by_atoms(filtered, LEVEL1)

In [212]:
len(chno)

183075

In [213]:
one_and_two = filter_by_atoms(filtered, LEVEL1.union(LEVEL2))

In [214]:
len(one_and_two)

348854

In [215]:
len(filtered)

385557

In [18]:
three = filter_by_atoms(filtered, LEVEL1 | LEVEL2 | LEVEL3)

In [19]:
len(three)

381387

In [20]:
three_cores = [r['reaction_core'] for r in three]

In [22]:
with open('cores.txt', 'w') as fn:
    for r in three_cores:
        fn.write(f"{r}\n")

In [23]:
tokenized = [smi_tokenizer(r) for r in three_cores]

In [24]:
with open('cores_tokenized.txt', 'w') as fn:
    for r in tokenized:
        fn.write(f"{r}\n")

In [218]:
four = filter_by_atoms(filtered, LEVEL1 | LEVEL2 | LEVEL3 | LEVEL4)

In [219]:
len(four)

384279

In [220]:
len(filtered)

385557

## Токенизация реакций и реакшн коров:

In [13]:
three_r = filter_reactions_by_atoms(filtered, LEVEL1 | LEVEL2 | LEVEL3)

In [14]:
len(three_r)

380733

In [80]:
def tokenize(reactions: list) -> list:
    from copy import deepcopy
    result = []
    cant_parse = []
    for reaction in tqdm_notebook(reactions):
        curr = deepcopy(reaction)
        try:
            curr['reaction_core'] = smi_tokenizer(curr['reaction_core'])
            curr['reactants'] = [smi_tokenizer(r) for r in curr['reactants']]
            curr['products'] = [smi_tokenizer(r) for r in curr['products']]
            result.append(curr)
        except:
            cant_parse.append(reaction)
    return result, cant_parse

In [81]:
tokenized, cp = tokenize(three_r)

In [92]:
def make_json(reactions: list) -> dict:
    res = {}
    for r in reactions:
        key = r['reaxys_id']
        res[key] = r
    return res

In [93]:
json_res = make_json(tokenized)

In [97]:
with open('tokenized_reactions.txt', 'w') as fstream:
    for r in tokenized:
        json.dump(r, fstream)
        fstream.write('\n')

In [3]:
test = []
with open('tokenized_reactions.txt') as fstream:
    for line in fstream:
        test.append(json.loads(line))

In [4]:
len(test)

380733

In [ ]:
reac